# RFM Analizi ile Müşteri Segmentasyonu

## İş Problemi

Online ayakkabı mağazası olan FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranışlardaki öbeklenmelere göre gruplar oluşturulacak.

## Veri Seti Hikayesi

Veri seti Flo’dan son alışverişlerini 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

### Görev 1: Veriyi Anlama ve Hazırlama

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width',1000)

In [2]:
# Adım 1: flo_data_20K.csv verisini okuyunuz.Dataframe’in kopyasını oluşturunuz.
flo_ = pd.read_csv("flo_data_20k.csv")
flo = flo_.copy()

In [3]:
# Adım 2:
# a. İlk 10 gözlem
flo.head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.00,2.00,150.87,49.99,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.00,1.00,59.99,315.94,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.00,1.00,49.99,113.64,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.00,2.00,120.48,934.21,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.00,1.00,69.98,95.98,"[KADIN, AKTIFSPOR]"


In [4]:
flo.shape

(19945, 12)

In [5]:
# b. Değişken isimleri
flo.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline', 'order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_offline', 'customer_value_total_ever_online', 'interested_in_categories_12'], dtype='object')

In [6]:
# c. Betimsel İstatistik
flo.describe()

,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online
count,19945.00,19945.00,19945.00,19945.00
mean,3.11,1.91,253.92,497.32
std,4.23,2.06,301.53,832.60
min,1.00,1.00,10.00,12.99
25%,1.00,1.00,99.99,149.98
50%,2.00,1.00,179.98,286.46
75%,4.00,2.00,319.97,578.44
max,200.00,109.00,18119.14,45220.13


In [7]:
flo.describe([0.25, 0.50, 0.75, 0.90, 0.95,0.99]).T

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,7.00,10.00,20.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,4.00,4.00,7.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,519.95,694.22,1219.95,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,1082.04,1556.73,3143.81,45220.13


In [8]:
# d. Boş değer
flo.isnull().sum().sum()

0

In [9]:
flo.isnull().sum()

,0
master_id,0
order_channel,0
last_order_channel,0
first_order_date,0
last_order_date,0
last_order_date_online,0
last_order_date_offline,0
order_num_total_ever_online,0
order_num_total_ever_offline,0
customer_value_total_ever_offline,0


In [10]:
# e. Değişken tipleri incelemesi
flo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [11]:
# Adım 3: Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
# alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.
flo["order_num_total"] = flo["order_num_total_ever_online"] + flo["order_num_total_ever_offline"]
flo["customer_value_total"] = flo["customer_value_total_ever_offline"] + flo["customer_value_total_ever_online"]
print(flo["order_num_total"].head())
print(flo["customer_value_total"].head())

0    5.00
1   21.00
2    5.00
3    2.00
4    2.00
Name: order_num_total, dtype: float64
0    939.37
1   2013.55
2    585.32
3    121.97
4    209.98
Name: customer_value_total, dtype: float64


In [12]:
# Adım 4: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
flo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [14]:
flo["first_order_date"] = flo["first_order_date"].apply(pd.to_datetime)
flo["last_order_date"] = flo["last_order_date"].apply(pd.to_datetime)
flo["last_order_date_online"] = flo["last_order_date_online"].apply(pd.to_datetime)
flo["last_order_date_online"] = flo["last_order_date_online"].apply(pd.to_datetime)

In [15]:
flo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  object        
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [17]:
# Adım 5: Alışveriş kanallarındaki müşteri sayısının, toplam alınan ürün sayısının ve toplam harcamaların dağılımına bakınız.

flo.groupby("order_channel").agg({"master_id":"count",
                                 "order_num_total":"sum",
                                 "customer_value_total":"sum"})

,master_id,order_num_total,customer_value_total
order_channel,,,
Android App,9495,52269.00,7819062.76
Desktop,2735,10920.00,1610321.46
Ios App,2833,15351.00,2525999.93
Mobile,4882,21679.00,3028183.16


In [20]:
# Adım 6: En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.
flo.sort_values("customer_value_total", ascending=False).head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00,1.00,130.49,36687.80,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00,36818.29
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00,1.00,1263.76,32654.34,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00,33918.10
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00,1.00,538.94,30688.47,"[ERKEK, KADIN, AKTIFSPOR]",11.00,31227.41
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00,3.00,18119.14,2587.20,[AKTIFSPOR],4.00,20706.34


In [22]:
# Adım 7: En fazla siparişi veren ilk 10 müşteriyi sıralayınız.
flo.sort_values("order_num_total", ascending=False).head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00,2.00,684.97,45220.13,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00,45905.10
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00,1.00,49.99,12232.25,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00,12282.24
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00,109.00,10239.46,143.98,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00,10383.44
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00,87.00,8432.25,139.98,[ERKEK],88.00,8572.23
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00,81.00,3997.55,242.81,[ERKEK],83.00,4240.36


In [23]:
# Adım 8: Veri ön hazırlık sürecini fonksiyonlaştırınız.
def data_preparation (dataframe):
    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    return dataframe

### Görev 2: RFM Metriklerinin Hesaplanması

In [25]:
flo["last_order_date"].max()
analysis_date = dt.datetime(2021,6,1)
rfm = pd.DataFrame()
rfm["customer_id"] = flo["master_id"]
rfm["recency"] = (analysis_date - flo["last_order_date"]).dt.days
rfm["frequency"] = flo["order_num_total"]
rfm["monetary"] = flo["customer_value_total"]

In [26]:
rfm.head()

,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98


### Görev 3: RF Skorunun Hesaplanması

In [27]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))
rfm["RFM_SCORE"].head()

,RFM_SCORE
0,344
1,355
2,243
3,311
4,311


### Görev 4: RF Skorunun Segment Olarak Tanımlanması

In [29]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))
rfm["RF_SCORE"].head()

,RF_SCORE
0,34
1,35
2,24
3,31
4,31


In [30]:
seg_map = {
    r"[1-2][1-2]" : "hibernating",
    r"[1-2][3-4]" : "at_risk",
    r"[1-2]5" : "cant_loose",
    r"3[1-2]" : "about_to_sleep",
    r"33" : "need_attention",
    r"[3-4][4-5]" : "loyal_customers",
    r"41" : "promising",
    r"51" : "new_customers",
    r"[4-5][2-3]" : "potential_loyalists",
    r"5[4-5]" : "champions"
}
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00,939.37,3,4,4,344,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105,21.00,2013.55,3,5,5,355,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186,5.00,585.32,2,4,3,243,24,at_risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135,2.00,121.97,3,1,1,311,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86,2.00,209.98,3,1,1,311,31,about_to_sleep


### Görev 5: Aksiyon Zamanı !

In [31]:
# Adım 1: Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.
rfm.groupby("segment")[["recency","frequency","monetary"]].agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       113.79  1629      2.40  1629   359.01  1629
at_risk              241.61  3131      4.47  3131   646.61  3131
cant_loose           235.44  1200     10.70  1200  1474.47  1200
champions             17.11  1932      8.93  1932  1406.63  1932
hibernating          247.95  3604      2.39  3604   366.27  3604
loyal_customers       82.59  3361      8.37  3361  1216.82  3361
need_attention       113.83   823      3.73   823   562.14   823
new_customers         17.92   680      2.00   680   339.96   680
potential_loyalists   37.16  2938      3.30  2938   533.18  2938
promising             58.92   647      2.00   647   335.67   647

In [34]:
# Adım 2: RFM analizi yardımıyla aşağıda verilen 2 case için ilgili profildeki müşterileri bulun ve müşteri id'lerini csv olarak kaydediniz.
# a.
target_customer_id = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["customer_id"]
target_customer_id.head()

,customer_id
0,cc294636-19f0-11eb-8d74-000d3a38a36f
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11,c2e15af2-9eed-11e9-9897-000d3a38a36f
14,241f0ad0-afb5-11e9-9757-000d3a38a36f


In [38]:
customer_id = flo[(flo["master_id"].isin(target_customer_id)) & (flo["interested_in_categories_12"].str.contains("KADIN")) ]["master_id"]
customer_id.head()

,master_id
0,cc294636-19f0-11eb-8d74-000d3a38a36f
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f
11,c2e15af2-9eed-11e9-9897-000d3a38a36f
33,fb840306-1219-11ea-a001-000d3a38a36f


In [39]:
customer_id.to_csv("targer_customers.csv", index=False)

In [40]:
# b.
target_customer_id2 = rfm[rfm["segment"].isin(["cant_loose","hibernating","new_customers"])]["customer_id"]
target_customer_id2.head()

,customer_id
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f
9,1143f032-440d-11ea-8b43-000d3a38a36f
10,ae608ece-c9d8-11ea-a31e-000d3a38a36f
15,13ed97a4-b167-11e9-89fa-000d3a38a36f
19,2730793e-3908-11ea-85d6-000d3a38a36f


In [43]:
customer_id2 = flo[(flo["master_id"].isin(target_customer_id2)) & ((flo["interested_in_categories_12"].str.contains("ERKEK"))|(flo["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
customer_id2.head()

,master_id
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f
10,ae608ece-c9d8-11ea-a31e-000d3a38a36f
15,13ed97a4-b167-11e9-89fa-000d3a38a36f
19,2730793e-3908-11ea-85d6-000d3a38a36f
21,7b289956-d691-11e9-93bc-000d3a38a36f


In [44]:
customer_id2.to_csv("targer_customer2.csv", index=False)